In [12]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, roc_curve, auc
import os

# from plots import plot_confusion_matrix, plot_roc_curve

In [3]:
model_path = '../models/deepfake_detection_models/faces_trained_model.h5'
model = load_model(model_path)

2024-04-08 20:22:51.812785: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14781 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0000:d8:00.0, compute capability: 7.0


In [14]:
test_dir = '../test_dataset'  # Adjust the path as necessary
images = []
labels = []

for idx, label in enumerate(["REAL", "FAKE"]):
    test_path = os.path.join(test_dir, label)
    for img_path in list(map(lambda x: os.path.join(test_path, x), os.listdir(test_path))):
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))  # Resize to match model input, adjust as necessary
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert color from BGR to RGB
        images.append(img)
        labels.append(idx)

images = np.array(images)
# The preprocess_input function expects float inputs
images = preprocess_input(images.astype('float32'))

labels = np.array(labels)

# Predict
predictions = model.predict(images)

247/247 [==============================] - 38s 128ms/step


In [15]:
# Convert predictions to label indices (0 or 1) based on a threshold (e.g., 0.5)
predicted_classes = (predictions > 0.5).astype('int').reshape(-1)

In [16]:
accuracy = accuracy_score(labels, predicted_classes)
precision = precision_score(labels, predicted_classes)
recall = recall_score(labels, predicted_classes)
print(f"Accuracy: {accuracy}, Precision: {precision}, Recall: {recall}")

Accuracy: 0.8346456692913385, Precision: 0.8763102725366876, Recall: 0.9376401973979362


In [ ]:
# Output directory for plots
output_dir = 'evaluation_plots'
os.makedirs(output_dir, exist_ok=True)

# Confusion Matrix Plot
plot_confusion_matrix(labels, predicted_classes, output_dir, "Confusion Matrix")

# ROC Curve Plot
fpr, tpr, _ = roc_curve(labels, predictions)
roc_auc = auc(fpr, tpr)
print(f"AUC: {roc_auc}")
plot_roc_curve(labels, predictions.ravel(), output_dir, "ROC Curve")